# Day5_1: 이상치 및 데이터 정제 (Outliers & Data Cleaning) - 정답

## 📚 학습 목표

**Part 1: 기초 - 이상치 탐지**
1. IQR(사분위수) 방법으로 이상치 탐지하기
2. Z-score 방법으로 이상치 탐지하기
3. 박스플롯(px.box)으로 이상치 시각화하기
4. 산점도(px.scatter)로 이상치 확인하기
5. 이상치 제거 및 대체 전략 이해하기

**Part 2: 심화 - 데이터 정제**
1. astype()으로 데이터 타입 변환하기
2. pd.to_datetime()으로 날짜 데이터 변환하기
3. replace()로 잘못된 값 대체하기
4. Min-Max 정규화 및 Z-score 스케일링 적용하기

---

## 🎯 실습 퀴즈 정답

---

### Q1. IQR로 이상치 찾기 ⭐

**문제**: 다음 월별 방문자 수 데이터에서 IQR 방법으로 이상치를 찾으세요.

In [ ]:
# Q1 정답
import pandas as pd
import numpy as np

visitors = pd.DataFrame({
    '월': ['1월', '2월', '3월', '4월', '5월', '6월'],
    '방문자수': [1000, 1050, 1020, 5000, 1080, 1040]
})

# 1. Q1, Q3, IQR 계산
Q1 = visitors['방문자수'].quantile(0.25)
Q3 = visitors['방문자수'].quantile(0.75)
IQR = Q3 - Q1

# 2. 이상치 범위 계산
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# 3. 이상치 필터링
outliers = visitors[(visitors['방문자수'] < lower_bound) | (visitors['방문자수'] > upper_bound)]

print(f"정상 범위: [{lower_bound:.1f}, {upper_bound:.1f}]")
print("\n이상치:")
print(outliers)

In [ ]:
# 테스트 케이스
assert len(outliers) == 1, "이상치는 1개여야 합니다"
assert outliers.iloc[0]['월'] == '4월', "4월이 이상치입니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
IQR 방법의 3단계: Q1/Q3 계산 → IQR 계산 → 범위 밖 값 필터링

**핵심 개념**:
- `quantile(0.25)`: 25번째 백분위수 (Q1)
- `quantile(0.75)`: 75번째 백분위수 (Q3)
- 이상치 범위: `[Q1 - 1.5*IQR, Q3 + 1.5*IQR]`
- 논리 연산: `|` (OR)로 상한/하한 초과 값 모두 찾기

**대안 솔루션**:
```python
# 함수로 만들기
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    return df[(df[column] < Q1 - 1.5*IQR) | (df[column] > Q3 + 1.5*IQR)]
```

**흔한 실수**:
- ❌ `&` 대신 `|` 사용: AND가 아니라 OR 연산!
- ❌ 괄호 누락: `(조건1) | (조건2)` 형태로

**실무 팁**:
IQR은 정규분포 가정이 필요 없어 가장 범용적입니다. 실무에서 90% 이상 이 방법 사용!

---

### Q2. Z-score 이상치 탐지 ⭐⭐

**문제**: 다음 주문액 데이터에서 Z-score 방법으로 이상치를 찾으세요 (|Z-score| > 2).

In [ ]:
# Q2 정답
orders = pd.DataFrame({
    '주문액': [30000, 35000, 32000, 150000, 33000, 31000, 34000]
})

# 1. 평균과 표준편차 계산
mean = orders['주문액'].mean()
std = orders['주문액'].std()

# 2. Z-score 계산
orders['Z-score'] = (orders['주문액'] - mean) / std

# 3. |Z-score| > 2인 이상치 찾기
outliers = orders[orders['Z-score'].abs() > 2]

print(f"평균: {mean:,.0f}원")
print(f"표준편차: {std:,.0f}원")
print("\n이상치:")
print(outliers)

In [ ]:
# 테스트
assert len(outliers) == 1, "이상치는 1개여야 합니다"
assert outliers.iloc[0]['주문액'] == 150000, "150000원이 이상치입니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
Z-score = (값 - 평균) / 표준편차 공식을 적용하여, 절댓값이 임계값(2 또는 3)을 초과하는 값을 찾습니다.

**핵심 개념**:
- `mean()`: 평균 계산
- `std()`: 표준편차 계산
- `abs()`: 절댓값 (양수/음수 모두 탐지)
- Z-score 의미: 평균으로부터 몇 표준편차 떨어져 있는가?

**대안 솔루션**:
```python
# scipy 라이브러리 활용
from scipy import stats
z_scores = np.abs(stats.zscore(orders['주문액']))
outliers = orders[z_scores > 2]
```

**흔한 실수**:
- ❌ `abs()` 누락: 음수 Z-score도 이상치!
- ❌ 임계값 혼동: 일반적으로 2 또는 3 사용

**실무 팁**:
Z-score는 정규분포 가정이 필요합니다. 데이터가 심하게 치우쳐 있으면 IQR 사용!

---

### Q3. 박스플롯으로 시각화 ⭐⭐

**문제**: 다음 데이터를 박스플롯으로 시각화하여 이상치를 파악하세요.

In [ ]:
# Q3 정답
import plotly.express as px

data = pd.DataFrame({
    '제품': ['A', 'B', 'C', 'D', 'E', 'F'],
    '가격': [10000, 12000, 11000, 50000, 10500, 11500]
})

# 박스플롯 생성
fig = px.box(data, y='가격',
             title='제품 가격 박스플롯 (이상치 자동 표시)',
             labels={'가격': '가격 (원)'},
             points='all')  # 모든 데이터 포인트 표시

fig.update_layout(height=500)
fig.show()

### 💡 풀이 설명

**접근 방법**:
`px.box()`는 IQR 방법을 자동으로 적용하여 박스 밖의 점을 이상치로 표시합니다.

**핵심 개념**:
- `px.box(df, y='컬럼')`: 세로 박스플롯
- `points='all'`: 모든 데이터 포인트를 점으로 표시
- 박스 밖의 점 = 이상치 (IQR 기준)

**대안 솔루션**:
```python
# 가로 박스플롯
fig = px.box(data, x='가격')

# 여러 그룹 비교
fig = px.box(data, x='카테고리', y='가격')
```

**흔한 실수**:
- ❌ x/y 축 혼동: 숫자 컬럼을 y축에
- ❌ `points` 옵션 누락: 점 없으면 박스만 보임

**실무 팁**:
박스플롯은 이상치를 **한눈에** 파악하는 최고의 도구! 데이터 탐색 시 가장 먼저 그려보세요.

---

### Q4. 문자열을 숫자로 변환 ⭐⭐

**문제**: 쉼표가 포함된 가격 문자열을 정수로 변환하고 평균을 구하세요.

In [ ]:
# Q4 정답
products = pd.DataFrame({
    '제품명': ['노트북', '마우스', '키보드'],
    '가격': ['1,200,000', '30,000', '80,000']
})

# 1. 쉼표 제거
products['가격_정제'] = products['가격'].str.replace(',', '')

# 2. 정수로 변환
products['가격_정제'] = products['가격_정제'].astype(int)

# 3. 평균 계산
avg_price = products['가격_정제'].mean()

print(products)
print(f"\n평균 가격: {avg_price:,.0f}원")

In [ ]:
# 테스트
assert products['가격_정제'].dtype == 'int64', "정수 타입이어야 합니다"
assert avg_price == 436666.666..., "평균 계산 오류"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
문자열 정제 → 타입 변환 2단계로 처리합니다.

**핵심 개념**:
- `str.replace(',', '')`: 문자열에서 쉼표 제거
- `astype(int)`: 문자열을 정수로 변환
- 체이닝 가능: `df['col'].str.replace().astype()`

**대안 솔루션**:
```python
# 한 줄로
products['가격_정제'] = products['가격'].str.replace(',', '').astype(int)

# apply 활용
products['가격_정제'] = products['가격'].apply(lambda x: int(x.replace(',', '')))
```

**흔한 실수**:
- ❌ `replace(',', '')` 대신 `str.replace(',', '')`: pandas는 `.str` 필요!
- ❌ 쉼표 제거 전 `astype()`: 변환 오류 발생

**실무 팁**:
웹 스크레이핑한 가격 데이터는 거의 항상 이 처리가 필요합니다!

---

### Q5. 날짜 변환 및 추출 ⭐⭐

**문제**: 주문일을 datetime으로 변환하고 '월' 정보를 추출하세요.

In [ ]:
# Q5 정답
sales = pd.DataFrame({
    '주문일': ['2024-01-15', '2024-02-20', '2024-03-10'],
    '매출': [1000, 1200, 980]
})

# 1. datetime으로 변환
sales['주문일'] = pd.to_datetime(sales['주문일'])

# 2. 월 추출
sales['월'] = sales['주문일'].dt.month

print(sales)
print(f"\n타입: {sales['주문일'].dtype}")

In [ ]:
# 테스트
assert sales['주문일'].dtype == 'datetime64[ns]', "datetime 타입이어야 합니다"
assert list(sales['월']) == [1, 2, 3], "월 추출 오류"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
`pd.to_datetime()` 변환 후 `.dt` 접근자로 날짜 요소 추출

**핵심 개념**:
- `pd.to_datetime()`: 문자열을 datetime으로 변환
- `.dt.month`: 월 추출 (1~12)
- `.dt.year`, `.dt.day`, `.dt.day_name()`: 다른 요소들

**대안 솔루션**:
```python
# 여러 정보 한 번에
sales['연도'] = sales['주문일'].dt.year
sales['월'] = sales['주문일'].dt.month
sales['일'] = sales['주문일'].dt.day
sales['요일'] = sales['주문일'].dt.day_name()
```

**흔한 실수**:
- ❌ `.dt` 누락: `sales['주문일'].month` (오류!)
- ❌ 변환 전 추출 시도: 문자열에는 `.dt` 불가

**실무 팁**:
시계열 분석 전에 **항상** datetime으로 변환! 월별/요일별 집계가 가능해집니다.

---

## Part 2 심화 퀴즈 정답

---

### Q6. 값 대체로 데이터 정제 ⭐⭐⭐

**문제**: 성별 표기를 통일하고, 999는 NaN으로 변경하세요.

In [ ]:
# Q6 정답
customers = pd.DataFrame({
    '이름': ['김철수', '이영희', '박민수'],
    '성별': ['남', 'F', 'M'],
    '나이': [30, 999, 25]
})

# 1. 성별 통일
customers['성별'] = customers['성별'].replace({
    '남': '남성',
    'M': '남성',
    '여': '여성',
    'F': '여성'
})

# 2. 999를 NaN으로
customers['나이'] = customers['나이'].replace(999, np.nan)

print(customers)
print(f"\n결측치 개수: {customers['나이'].isna().sum()}")

In [ ]:
# 테스트
assert customers['성별'].tolist() == ['남성', '여성', '남성'], "성별 변환 오류"
assert customers['나이'].isna().sum() == 1, "999가 NaN으로 변환되어야 함"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
`replace()` 딕셔너리로 여러 값을 한 번에 매핑하여 변환합니다.

**핵심 개념**:
- `replace({old1: new1, old2: new2})`: 딕셔너리로 다중 변환
- `replace(999, np.nan)`: 단일 값 변환
- 체이닝 가능

**대안 솔루션**:
```python
# map 활용
gender_map = {'남': '남성', 'M': '남성', '여': '여성', 'F': '여성'}
customers['성별'] = customers['성별'].map(gender_map)

# 조건부 변환
customers.loc[customers['나이'] == 999, '나이'] = np.nan
```

**흔한 실수**:
- ❌ `replace()` 대신 `str.replace()`: 문자열 메소드 아님!
- ❌ NaN 대신 문자열 'NaN': `np.nan` 또는 `pd.NA` 사용

**실무 팁**:
실무 데이터는 입력자마다 표기가 다릅니다. 분석 전 반드시 통일!

---

### Q7. Min-Max 정규화 ⭐⭐⭐

**문제**: 연봉과 나이를 0~1 범위로 정규화하세요.

In [ ]:
# Q7 정답
employees = pd.DataFrame({
    '직원명': ['김철수', '이영희', '박민수'],
    '연봉': [3000, 5000, 4000],
    '나이': [28, 35, 30]
})

# Min-Max 정규화 공식: (x - min) / (max - min)
for col in ['연봉', '나이']:
    min_val = employees[col].min()
    max_val = employees[col].max()
    employees[f'{col}_정규화'] = (employees[col] - min_val) / (max_val - min_val)

print(employees)

In [ ]:
# 테스트
assert employees['연봉_정규화'].min() == 0, "최소값은 0이어야 함"
assert employees['연봉_정규화'].max() == 1, "최대값은 1이어야 함"
assert employees['나이_정규화'].min() == 0, "최소값은 0이어야 함"
assert employees['나이_정규화'].max() == 1, "최대값은 1이어야 함"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
Min-Max 공식을 각 컬럼에 적용하여 0~1 범위로 변환합니다.

**핵심 개념**:
- 공식: `(x - min) / (max - min)`
- 최소값 → 0, 최대값 → 1
- 모든 값이 0~1 사이에 분포

**대안 솔루션**:
```python
# sklearn 활용 (더 간단)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
employees[['연봉_정규화', '나이_정규화']] = scaler.fit_transform(employees[['연봉', '나이']])
```

**흔한 실수**:
- ❌ 분모 `(max - min)` 누락: 단순 빼기만 하면 안 됨
- ❌ 제로 나누기: min==max일 때 처리 필요

**실무 팁**:
딥러닝 입력 데이터는 거의 항상 0~1 정규화합니다! 학습 안정성과 속도 향상.

---

### Q8. 산점도로 이상치 시각화 ⭐⭐⭐⭐

**문제**: 광고비와 매출 산점도를 그리고, 이상치를 빨간색으로 표시하세요.

In [ ]:
# Q8 정답
ad_sales = pd.DataFrame({
    '광고비': [100, 200, 300, 400, 500, 250],
    '매출': [500, 1000, 1500, 2000, 2500, 100]  # 마지막이 이상치
})

# 이상치 여부 컬럼 추가
ad_sales['이상치'] = ['정상'] * 5 + ['이상치']

# 색상 매핑 산점도
fig = px.scatter(ad_sales, x='광고비', y='매출',
                 color='이상치',
                 color_discrete_map={'정상': 'blue', '이상치': 'red'},
                 title='광고비 vs 매출 (이상치 표시)',
                 labels={'광고비': '광고비 (만원)', '매출': '매출 (만원)'},
                 size=[10]*5 + [20])  # 이상치 크기 강조

fig.show()

### 💡 풀이 설명

**접근 방법**:
이상치 여부를 별도 컬럼으로 만들고, `color` 파라미터로 색상 구분합니다.

**핵심 개념**:
- `color='이상치'`: 컬럼 값에 따라 자동 색상 지정
- `color_discrete_map`: 색상 수동 매핑
- `size`: 점 크기 리스트로 지정

**대안 솔루션**:
```python
# 직접 색상 리스트 지정
colors = ['blue']*5 + ['red']
fig = px.scatter(ad_sales, x='광고비', y='매출')
fig.update_traces(marker=dict(color=colors, size=[10]*5 + [20]))
```

**흔한 실수**:
- ❌ `color` 없이 `update_traces()`: 복잡함
- ❌ 색상 매핑 오타: 'blue' 정확히

**실무 팁**:
산점도에서 이상치는 시각적으로 즉시 파악 가능! 2변수 관계 분석 시 필수.

---

### Q9. Z-score 표준화 ⭐⭐⭐⭐

**문제**: 매출 데이터를 Z-score로 표준화하고, 평균과 표준편차를 확인하세요.

In [ ]:
# Q9 정답
monthly_sales = pd.DataFrame({
    '매출': [1000, 1200, 1100, 1300, 1050, 1150]
})

# 1. 평균과 표준편차 계산
mean = monthly_sales['매출'].mean()
std = monthly_sales['매출'].std()

# 2. Z-score 표준화
monthly_sales['매출_표준화'] = (monthly_sales['매출'] - mean) / std

print(monthly_sales)

# 3. 표준화 후 통계량 확인
new_mean = monthly_sales['매출_표준화'].mean()
new_std = monthly_sales['매출_표준화'].std()

print(f"\n표준화 후 평균: {new_mean:.10f} (≈ 0)")
print(f"표준화 후 표준편차: {new_std:.10f} (≈ 1)")

In [ ]:
# 테스트
assert abs(new_mean) < 1e-10, "평균은 0에 가까워야 함"
assert abs(new_std - 1.0) < 0.01, "표준편차는 1에 가까워야 함"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
Z-score 공식을 적용하여 평균 0, 표준편차 1로 변환합니다.

**핵심 개념**:
- 공식: `(x - 평균) / 표준편차`
- 변환 후 평균 ≈ 0 (매우 작은 값)
- 변환 후 표준편차 = 1
- **이상치에 민감**: 평균/표준편차가 이상치 영향 받음

**대안 솔루션**:
```python
# sklearn 활용
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
monthly_sales['매출_표준화'] = scaler.fit_transform(monthly_sales[['매출']])
```

**흔한 실수**:
- ❌ 표준편차 대신 분산 사용
- ❌ 평균이 정확히 0 기대: 부동소수점 오차 있음

**실무 팁**:
전통적인 ML(선형회귀, SVM 등)은 Z-score 표준화를 선호합니다!

---

### Q10. 종합 데이터 정제 파이프라인 ⭐⭐⭐⭐⭐

**문제**: 다음 더러운 데이터를 완전히 정제하세요.

In [ ]:
# Q10 정답
dirty = pd.DataFrame({
    '주문일': ['2024-01-15', '2024-02-20', '2024-03-10'],
    '주문액': ['100,000', '200,000', '150,000'],
    '성별': ['M', 'F', 'M'],
    '나이': [999, 30, 25]
})

print("=== 원본 데이터 ===")
print(dirty)
print(f"\n타입:\n{dirty.dtypes}")

# 정제 파이프라인
clean = dirty.copy()

# 1. 주문일 datetime 변환 + 월 추출
clean['주문일'] = pd.to_datetime(clean['주문일'])
clean['월'] = clean['주문일'].dt.month

# 2. 주문액 쉼표 제거 + 정수 변환
clean['주문액'] = clean['주문액'].str.replace(',', '').astype(int)

# 3. 성별 통일
clean['성별'] = clean['성별'].replace({'M': '남성', 'F': '여성'})

# 4. 나이 999를 NaN으로
clean['나이'] = clean['나이'].replace(999, np.nan)

# 5. 주문액 Min-Max 정규화
min_val = clean['주문액'].min()
max_val = clean['주문액'].max()
clean['주문액_정규화'] = (clean['주문액'] - min_val) / (max_val - min_val)

print("\n=== 정제된 데이터 ===")
print(clean)
print(f"\n타입:\n{clean.dtypes}")

In [ ]:
# 테스트
assert clean['주문일'].dtype == 'datetime64[ns]', "날짜 타입 오류"
assert clean['주문액'].dtype == 'int64', "주문액 타입 오류"
assert clean['성별'].tolist() == ['남성', '여성', '남성'], "성별 변환 오류"
assert clean['나이'].isna().sum() == 1, "999 변환 오류"
assert clean['주문액_정규화'].min() == 0, "정규화 최소값 오류"
assert clean['주문액_정규화'].max() == 1, "정규화 최대값 오류"
print("✅ 모든 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
5단계 정제 파이프라인을 순차적으로 적용합니다.

**핵심 개념**:
1. **날짜 변환**: `pd.to_datetime()` + `.dt.month`
2. **문자열→숫자**: `str.replace()` + `astype()`
3. **값 대체**: `replace()` 딕셔너리
4. **결측 처리**: `replace(999, np.nan)`
5. **정규화**: Min-Max 공식

**대안 솔루션**:
```python
# 체이닝으로 한 번에
clean = (dirty
    .assign(주문일=lambda x: pd.to_datetime(x['주문일']))
    .assign(월=lambda x: x['주문일'].dt.month)
    .assign(주문액=lambda x: x['주문액'].str.replace(',', '').astype(int))
    .assign(성별=lambda x: x['성별'].replace({'M': '남성', 'F': '여성'}))
    .assign(나이=lambda x: x['나이'].replace(999, np.nan))
)
```

**흔한 실수**:
- ❌ 순서 중요: 타입 변환 전에 문자열 처리!
- ❌ 원본 수정: `copy()` 필수
- ❌ 중간 결과 미확인: 각 단계마다 `print()`로 확인

**실무 팁**:
실무 데이터 정제는 항상 **파이프라인 형태**로 작성합니다. 재사용과 디버깅이 쉬워집니다!

---

## 📊 학습 정리

### Part 1: 이상치 탐지 핵심 요약

| 방법 | 핵심 개념 | 실무 활용 |
|------|----------|----------|
| IQR | Q1 - 1.5*IQR ~ Q3 + 1.5*IQR | 가장 범용적, 분포 가정 불필요 |
| Z-score | \|Z\| > 3 (평균으로부터 3σ) | 정규분포 가정, 통계 분석용 |
| 박스플롯 | px.box() | 이상치 시각적 파악 |
| 산점도 | px.scatter() | 2변수 관계에서 이상치 탐지 |

### Part 2: 데이터 정제 핵심 요약

| 기법 | 문법 | 언제 쓰나? |
|------|------|----------|
| 타입 변환 | `astype(int)`, `str.replace()` | 문자열 숫자를 실제 숫자로 |
| 날짜 변환 | `pd.to_datetime()` | 시계열 분석, 날짜 연산 |
| 값 대체 | `replace({old: new})` | 표기 통일, 오타 수정 |
| Min-Max 정규화 | `(x - min) / (max - min)` | 0~1 범위, 딥러닝 전처리 |
| Z-score 표준화 | `(x - mean) / std` | 평균 0/표준편차 1, ML 전처리 |

### 💡 실무 팁

1. **이상치 처리 순서**: 탐지 → 시각화 → 원인 파악 → 제거/대체 결정
2. **IQR vs Z-score**: 분포 모르면 IQR, 정규분포면 Z-score
3. **타입 변환**: 항상 `errors='coerce'` 옵션으로 안전하게
4. **정규화 vs 표준화**: 딥러닝은 Min-Max, 전통 ML은 Z-score
5. **전후 비교**: 정제 전후 히스토그램/박스플롯 필수!